# Consolidated Scraped Data 

## Introduction
This notebook consolidates data is collected from three sources ([The-Numbers](https://www.the-numbers.com/), [IMDb](https://www.imdb.com/) and [Wikipedia](https://www.wikipedia.org)). Links to notebooks that collect/scrape data from [The-Numbers](the_numbers_scraper.ipynb) and [IMDb](imdb_dataset_download.ipynb) are provided.

In this notebook, datasets from the two sources are merged using movie title and release year. To do this, spaces and special characters are stripped from the movie titles. 

- For entries that do not match, these additional steps were taken:
    - Remove _'the'_ and _'and'_ from titles on the unmatched datasets
    - Match using title and select the nearest year of release (atmost ± 5 years apart)
    - For The-Numbers dataset with unknown dataset, match only those with single rows after being merged

In the next step, movie titles are scraped from Wikipedia for names of the companies that distribute the movies. The final dataframe is also merged with other tables from IMDb datasets using the unique movie id. 

## The code

In [1]:
#First, install the wikipedia library 
#!pip install wikipedia

In [2]:
#Import important libraries 
import requests 
import pandas as pd 
import re
from bs4 import BeautifulSoup
import wikipedia

In [3]:
#Load the-numbers dataset saved to disk. Run the_numbers_scraper.ipynb first
tn_df = pd.read_csv("zippedData/tn_movie_budgets_updated.csv.gz",compression='gzip')

#Create 'startYear' column in tn_df, with similar name to imdb_df
tn_df['startYear'] = tn_df['release_date'].map(lambda x: x.split()[-1])

In [4]:
#Load IMDb dataset. For title matching, title.basics.tsv.gz file is used. Run imdb_dataset_download.ipynb prior to this code
imdb_df = pd.read_csv("zippedData/title.basics.tsv.gz",compression='gzip',delimiter='\t',low_memory=False)

#Select rows with 'titleType' == 'movie'
imdb_df = imdb_df[imdb_df['titleType']=='movie']

In [5]:
#Set '\N' values to 'Unknown' (similar to tn_df) and replace numbers in 'startYear' to strings 
imdb_df[imdb_df['startYear']=='\\N']='Unknown'
imdb_df['startYear'] = imdb_df['startYear'].astype(str)

In [6]:
#Assign columns to merge on, and columns to choose from imdb_df
cols = ['tconst','primaryTitle','originalTitle','isAdult','runtimeMinutes','genres','simple_title','startYear']

In [7]:
#Function strips away special characters and additional list of strings 
def prep_title(title, replace=False):
    words = ['the','and']
    title = [x.lower() for x in title.split()]
    if replace:
        for word in words:
            if word in title:
                title.remove(word)
    return re.sub('[\W\_]','',''.join(title))

#Function that returns rows present only in df_1 but not in d_
def left_outer_merge(df_1,df_2,column):
    left_out = df_1.merge(df_2[column], how="left", on=column,indicator=True)
    left_out=left_out[left_out['_merge']=='left_only']
    return left_out.drop('_merge',axis=1)

In [8]:
#Create a column named 'simple_title' in 'tn_df' for comparing titles
tn_df['simple_title'] = tn_df['movie'].map(lambda x: prep_title(x))

Now we are ready to merge the two datasets. IMDb dataset has two columns for titles: original and primary. Therefore, titles are prepped from these two columns separately and merged with `tn_df`.

In [9]:
#Create a new dataframe called movies_df
movies_df = pd.DataFrame(columns=list(tn_df.columns)+cols[:-2])

In [10]:
#Function that preps the title and concatinates dataframes on orignal and primary titles
def concat_with_movies(df_1,df_2,
                       merge_cols=['simple_title','startYear'],
                       replace=False,
                       m_df=pd.DataFrame(),
                       cols=cols):
    #######################################################################
    #######################################################################
    #df_1: The first dataframe 
    #df_2: The second dataframe. df_1 and df_2 must share columns 
    #      specified by merge_cols
    #merge_cols: list of columns on which the df_1 and df_2
    #replace: boolean to replace 'the' and 'and' in the movie titles
    #m_df: a dummy dataframe returned by the function, set empty by default
    #cols: columns selected from the df_2
    df_1['simple_title'] = df_1['movie'].map(lambda x: prep_title(x,replace))
    for col in df_2.columns:
        if 'Title' in col:            
            df_2['simple_title'] = df_2[col].map(lambda x: prep_title(x,replace))
            df = pd.merge(df_1,df_2[cols],on=merge_cols)
            if m_df.empty:
                m_df =df
            else:
                m_df = pd.concat([m_df,df[m_df.columns]])
    return m_df.drop_duplicates()

In [11]:
#First merge tn_df and imdb_df based on title and release year
movies_df = concat_with_movies(tn_df,imdb_df)

In [12]:
#Find unmerged rows
un_merged_1 = left_outer_merge(tn_df,movies_df,'simple_title')

In [13]:
#Remove 'the','and' on the unmerged rows and try again
movies_df = pd.concat([movies_df,concat_with_movies(un_merged_1,imdb_df,replace=True)])

In [14]:
#Find missing rows
un_merged_2 = left_outer_merge(tn_df,movies_df,'simple_title')

Now, let's try to merge `un_merged_2` using titles and check if the release years in `tn_df` and `imdb_df` are within 5 years of each other 

In [15]:
#Function that returns string of years ± 5 years in 'un_merged_2' dataframe
def list_year(year,gap):
    try:
        return ','.join([str(int(year)+i) for i in range(-gap,gap+1)])
    except:
        return year

In [16]:
#Merge un_merged_2 and imdb_df on title alone first with replacements 
empty_df = pd.DataFrame(columns=list(tn_df.columns)+cols)
mer_2_titl = concat_with_movies(un_merged_2,imdb_df,
                                    replace=True,
                                    merge_cols='simple_title')

Note that `mer_2_titl` has two `startYear` columns. We now apply `list_year` function to `startYear_x` column and check if `startYear_y` is in `startYear_x`.

In [17]:
#Make a list of years ± 2 years in 'startYear_x'
gap = 3
mer_2_titl['startYear_x'] = mer_2_titl['startYear_x'].apply(lambda x: list_year(x,gap))

In [18]:
#Select values 'startYear_y' that are in 'startYear_x' and store them in column called 'isin'
mer_2_titl['isin'] = [x[0] in x[1] for x in zip(mer_2_titl['startYear_y'],mer_2_titl['startYear_x'])]
mer_2_titl = mer_2_titl[mer_2_titl['isin']]

#Select all columns except 'startYear_x' and 'isin'
mer_2_titl.drop(['startYear_x','isin'],axis=1,inplace=True)

#Rename startYear_y to simply startYear
mer_2_titl.rename(columns = {'startYear_y':'startYear'}, inplace = True)

Despite the effort, however, it's possible that one title might be merged with multiple titles from `imdb_df`. Therefore, simply groupinig the table based on `id` and then taking the first entry is going to solve this issue.  

In [19]:
#Take the first in grouped rows by id and then reset the index
mer_2_titl = mer_2_titl.groupby('id').first().reset_index()

Now `mer_2_titl` can be concatenated with `movies_df`. 

In [20]:
movies_df = pd.concat([movies_df,mer_2_titl])

Lastly, we match the titles with unknown release data in `tn_df` with `imdb_df` that do not appear in `movies_df`. 

In [21]:
#Select titles with unknown 'startYear' that are not movies_df
year_unk_df = left_outer_merge(tn_df[tn_df['startYear']=='Unknown'],movies_df,'simple_title')

#Merge 'year_unk_df' and 'imdb_df' on title alone
unk_merge = concat_with_movies(year_unk_df,imdb_df,
                               replace=True,
                               merge_cols='simple_title')

Similar to earlier, `unk_merge` will have `startYear_x` and `startYear_y`columns.

In [22]:
#Drop 'startYear_x' and rename 'startYear_y' to 'startYear'
unk_merge.drop('startYear_x',axis=1, inplace=True)
unk_merge.rename(columns = {'startYear_y':'startYear'}, inplace = True)

#Take the first in grouped rows by id and then reset the index as a precaution
unk_merge = unk_merge.groupby('id').first().reset_index()

In [23]:
#Merge unk_merge with movies_df
movies_df = pd.concat([movies_df,unk_merge])

As a last effort, we are going to merge the remaining dataset in `tn_df`.

In [24]:
remaining_df = left_outer_merge(tn_df,movies_df,'movie')
name_only = concat_with_movies(remaining_df,imdb_df,
                                  replace=True,
                                  merge_cols='simple_title')

To merge the remaining titles, the nearest values of `startYear_y` to `startYear_x` were chosen. 

In [25]:
#Identify the unique ids to be iterated
unique_ids = name_only['id'].unique()

#Cast years to int
name_only[['startYear_x','startYear_y']] = name_only[['startYear_x','startYear_y']].astype(int)

#Loop through each unique id, collect df with similar id, find the closest year by subtracting startYear_x from startYear_y
# and keep the closest value
df = name_only.copy()
for id_ in unique_ids:
    tmp = name_only[name_only['id']==id_]
    df_idx = tmp.index.tolist() 
    diff = [abs(i-j) for i,j in zip(tmp['startYear_x'],tmp['startYear_y'])]
    idx = df_idx[diff.index(min(diff))]
    df_idx.remove(idx)
    df.drop(df_idx,axis=0,inplace=True)    
name_only = df

#Change the years back to str
name_only[['startYear_x','startYear_y']] = name_only[['startYear_x','startYear_y']].astype(str)

In [26]:
#Drop 'startYear_x' and rename 'startYear_y' to 'startYear'
name_only.drop('startYear_x',axis=1, inplace=True)
name_only.rename(columns = {'startYear_y':'startYear'}, inplace = True)

#Take the first in grouped rows by id and then reset the index as a precaution
name_only = name_only.groupby('id').first().reset_index()

#Merge name_only with movies_df
movies_df = pd.concat([movies_df,name_only])

Again, as a precaution, we have to group `movies_df` by `id`, reset the index and take the first value. 

In [27]:
movies_df = movies_df.groupby('tconst').first().reset_index()

#Remove 'simple_title' column
movies_df.drop('simple_title',axis=1,inplace=True)

In [28]:
discarded_df = left_outer_merge(tn_df,movies_df,'movie')
print('{} titles were discarded.'.format(len(discarded_df)))

201 titles were discarded.


201 movies out of 6183 is not bad. We will now proceed to merging the `movies_df` with `title.crew.tsv.gz`,`title.principals.tsv.gz`, `title.ratings.tsv.gz` and `name.basics.tsv.gz`

Due to time constraint and constantly maxing out requests, Wikipedia scraper was not implemented. The idea was to search for each movie title using the `wikipedia` library like so:

In [ ]:
#Find the word 'Distributed by' in 'infobox vevent' table

def distributed_filler(soup):
    ### soup is BeautifulSoup object parsing an html page
    distrib = 'Unknown'
    info_table = soup.find('table',class_='infobox vevent')        
    if info_table!=None:            
        col_names = []
        for info in info_table.find_all('th'):
            col_names.append(info.get_text(strip=True))
            if 'Distributed by' in col_names:
                col_vals = info_table.find_all('td')
                idx = col_names.index('Distributed by')
                distrib = col_vals[idx].get_text(strip=True)
    return distrib

In [ ]:
#This code scrapes the distributor of movies in movies_df. 
#Multiple requests can result in time out. 

import time

wiki = 'https://en.wikipedia.org/wiki/'
movies_df['distributedBy'] = ''
for i,movie in movies_df[52:].iterrows():
    results = wikipedia.search(movie['movie']+' film '+movie['startYear'])
    try:
        page = wikipedia.page(results[0]).html()
        soup = BeautifulSoup(page,'html.parser')
    except:
        url = wiki+results[0]
        html_page = requests.get(url,timeout=5)
        soup = BeautifulSoup(html_page.content, 'html.parser')
    movies_df.loc[i,'distributedBy'] = distributed_filler(soup) 

Now that `movies_df` is ready, the remaining dataset can be loaded and joined using `tconst` column

In [29]:
#Add rating 
rating_df = pd.read_csv("zippedData/title.ratings.tsv.gz",compression='gzip',delimiter='\t',low_memory=False)
movies_df = pd.merge(movies_df,rating_df,on='tconst')

In [31]:
#Add principals for each movie 
princ_df = pd.read_csv("zippedData/title.principals.tsv.gz",compression='gzip',delimiter='\t')
princ_df = pd.merge(princ_df[['tconst','nconst','category']],movies_df['tconst'],on='tconst')
princ_df.drop_duplicates(inplace=True)

#Find names
names = pd.read_csv("zippedData/name.basics.tsv.gz",compression='gzip',delimiter='\t')
names = pd.merge(names[['nconst','primaryName']],princ_df['nconst'],on='nconst')
names.drop_duplicates(inplace=True)

#Merge principals and names
princ_df = pd.merge(princ_df,names,on='nconst')
princ_df.drop('nconst',axis=1,inplace=True)

#Pivot table by 'tconst' and 'category'
pv_prin_df = princ_df.groupby(['tconst','category'])['primaryName'].apply(list).reset_index()
pv_prin_df = pv_prin_df.pivot(index='tconst', columns='category', values='primaryName').reset_index()

#Merge pv_prin_df with movies_df
movies_df = pd.merge(movies_df,pv_prin_df,on='tconst')

In [39]:
movies_df = movies_df.sort_values('id')

,tconst,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,startYear,primaryTitle,originalTitle,...,archive_footage,archive_sound,cinematographer,composer,director,editor,producer,production_designer,self,writer
5507,tt4154796,1,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,797,800,564",2019,Avengers: Endgame,Avengers: Endgame,...,NaN,NaN,NaN,NaN,"[Anthony Russo, Joe Russo]",NaN,NaN,NaN,NaN,"[Stan Lee, Jack Kirby, Christopher Markus, Ste..."
4003,tt1298650,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,071,802","$1,045,713,802",2011,Pirates of the Caribbean: On Stranger Tides,Pirates of the Caribbean: On Stranger Tides,...,NaN,NaN,NaN,NaN,[Rob Marshall],NaN,NaN,NaN,NaN,"[Ted Elliott, Terry Rossio, Jay Wolpert, Stuar..."
5010,tt2395427,3,"Apr 22, 2015",Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,395,316,979",2015,Avengers: Age of Ultron,Avengers: Age of Ultron,...,NaN,NaN,NaN,NaN,[Joss Whedon],NaN,[Kevin Feige],NaN,NaN,"[Stan Lee, Jack Kirby, Joe Simon, Jim Starlin]"
5058,tt2488496,4,"Dec 16, 2015",Star Wars Episode VII: The Force Awakens,"$306,000,000","$936,662,225","$2,064,615,817",2015,Star Wars: Episode VII - The Force Awakens,Star Wars: Episode VII - The Force Awakens,...,NaN,NaN,NaN,NaN,[J.J. Abrams],NaN,"[Kathleen Kennedy, Bryan Burk]",NaN,NaN,"[George Lucas, Lawrence Kasdan, Michael Arndt]"
5506,tt4154756,5,"Apr 25, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,044,540,523",2018,Avengers: Infinity War,Avengers: Infinity War,...,NaN,NaN,NaN,NaN,"[Anthony Russo, Joe Russo]",NaN,NaN,NaN,NaN,"[Stan Lee, Jack Kirby, Christopher Markus, Ste..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5933,tt7837402,6179,Unknown,Red 11,"$7,000",$0,$0,2019,Red 11,Red 11,...,NaN,NaN,NaN,[Rebel Rodriguez],[Robert Rodriguez],NaN,NaN,NaN,NaN,[Racer Rodriguez]
1395,tt0154506,6180,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495",1998,Following,Following,...,NaN,NaN,NaN,[David Julyan],[Christopher Nolan],[Gareth Heal],[Emma Thomas],NaN,NaN,NaN
2980,tt0461832,6181,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338",2004,Return to the Land of Wonders,Return to the Land of Wonders,...,NaN,NaN,NaN,NaN,[Maysoon Pachachi],"[Anne Even, Gary Sims]",NaN,NaN,NaN,NaN
4820,tt2107644,6182,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0,2013,A Plague So Pleasant,A Plague So Pleasant,...,NaN,NaN,NaN,NaN,"[Benjamin Roberds, Jordan Reyes]",NaN,NaN,NaN,NaN,NaN
